<a href="https://colab.research.google.com/github/jorge774/Procesamiento_datos/blob/main/procesar_datos_v2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''procesar_datos.py
   Este programa hace lo siguiente:
   1.Reavisa la carpeta donde estan los archivos txt que genera janus luego de hacer adquisicion.
   2.Revisa y toma los archivos txt que no ha leido anteriormente.
   3.Toma el encabezado del archivo RunXX_list.txt con menor XX de todo el lote de archivos tomados en 2.
   4.Toma como referencia temporal el 'Run start time' del encabezado tomado en 3.
   5.Genera una estampa temporal para cada evento de todos los archivos RunXX_list.txt tomando como referencia a 4.
   6.Genera un archivo 'MuTe_20240304_11h07m01s.txt'
   7.Elimina aquellos eventos donde el conteo en todos los canales fue 0.
   8.Elimina los archivos RunXX_list.txt

   Los parametros de este programa son leidos en el archivo 'config.txt', los cuales son:
   1.Ruta absoluta donde quedan los archivos txt que genera janus luego de hacer adquisicion.
   2.Ruta absoluta donde quedaran los archivos 'MuTe_20240304_11h07m01s.txt'.
   3.Numero de canales usados en la adquision.
   4.Numero de lineas usadas en los encabezados de los archivos 'RunXX_list.txt'.

   En el archivo 'vars.txt' el programa escribe el nombre de los archivos 'RunXX_list.txt' que va procesando.
'''

In [ ]:
print('procesando...')
import os
import re
import time
from datetime import datetime
from datetime import timedelta
from pytz import timezone
import pandas as pd
ini_exe=datetime.now(timezone('America/Bogota'))
config_file=open(file='config.txt',mode='r');output_input_folders=config_file.read();config_file.close()
num_header_lines=int(output_input_folders.split('\n')[3].split('#')[0])#8
n_channels=int(output_input_folders.split('\n')[2].split('#')[0])#64 #numero de canales
folder_path_caen=output_input_folders.split('\n')[0].split('#')[0]
output_folder_path=output_input_folders.split('\n')[1].split('#')[0]

In [ ]:
#gestor de errores en los archivos run list

#eliminar archivos.dat

#ordenar los files_to_proces de acuerdo al run start time
def key_function(file,folder_path_caen):
    if re.search('list.txt',file):
       file_path=folder_path_caen+'/'+file;file_to_proces=open(file=file_path,mode='r')
       content=file_to_proces.readlines()[:10];file_to_proces.close()
       string_date_time=''
       for i in content[6].split()[4:-1]:i+=' ';string_date_time+=i
       #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
       ref_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')-timedelta(hours=5)
       ref_date_time=pd.Timestamp(str(ref_date_time))
       del content
    if re.search('Info.txt',file):
       file_path=folder_path_caen+'/'+file;file_to_proces=open(file=file_path,mode='r')
       content=file_to_proces.readlines()[:10];file_to_proces.close()
       ref_date_time=content[3].split()[2]+' '+content[3].split()[3]
       ref_date_time=pd.Timestamp(datetime.strptime(ref_date_time,'%d/%m/%Y %H:%M'))
       del content
    #ignorar folders y archivos que no sean de info o corrida
    if not re.search('list.txt',file) and not re.search('Info.txt',file):ref_date_time=pd.Timestamp('2030-01-01 00:00:00')
    return ref_date_time

#verificar que un archivo de corrida tenga informacion de todos los canales en todas las ventanas de adquision
def check_arvhivo_corrida(file,folder_path_caen,num_header_lines,n_channels):
    check=True#archivo de corrida sin errores
    file_path=folder_path_caen+'/'+file;file_to_proces=open(file=file_path,mode='r')
    data_and_header=file_to_proces.readlines();file_to_proces.close()
    ini_index=num_header_lines+1
    end_index=num_header_lines+n_channels+1
    while ini_index<(len(data_and_header)-1):
          #recorrer todos los data blocks de un archivo Runxx_list.txt
          data_block=data_and_header[ini_index:end_index]
          if len(data_block[0].split())!=5:
             check=False#archivo de corrida con errores
             os.remove(file_path)#eliminar archivo runxxlits.txt corrupto
             print('archivo corrupto eliminado')
             break
          ini_index=end_index
          end_index=ini_index+n_channels
    del data_and_header
    return check

In [ ]:
folder_path_vars='vars.txt'
##ini main loop
vars_file=open(file=folder_path_vars,mode='r')
processed_file_list=vars_file.read()
vars_file.close()
current_file_list=os.listdir(folder_path_caen)

if processed_file_list:
   #processed_file_list is not empty
   files_to_proces=list(set(current_file_list)-set(processed_file_list.split('\n')))#archivos que esten en folder_path_caen pero que no esten en el vars file
else:
   #processed_file_list is empty
   files_to_proces=current_file_list

files_to_proces.sort(key=lambda file:key_function(file,folder_path_caen))#ordenar los files_to_proces de acuerdo al run start time
if files_to_proces:
    #files_to_proces is not empty
    count_files_run_list_processed=0
    #'MuTe_20240304_11h07m01s'
    output_file_path=''
    gmt_5_datetime=datetime.now(timezone('America/Bogota'))
    cur_h='0'+str(gmt_5_datetime.hour) if gmt_5_datetime.hour<10 else str(gmt_5_datetime.hour)
    cur_day='0'+str(gmt_5_datetime.day) if gmt_5_datetime.day<10 else str(gmt_5_datetime.day)
    cur_mon='0'+str(gmt_5_datetime.month) if gmt_5_datetime.month<10 else str(gmt_5_datetime.month)
    cur_min='0'+str(gmt_5_datetime.minute) if gmt_5_datetime.minute<10 else str(gmt_5_datetime.minute)
    cur_sec='0'+str(gmt_5_datetime.second) if gmt_5_datetime.second<10 else str(gmt_5_datetime.second)
    name='MuTe_'+str(gmt_5_datetime.year)+cur_mon+cur_day+'_'+cur_h+'h'+cur_min+'m'+cur_sec+'s'+'.txt'
    for file in files_to_proces:
        if re.search('list.txt',file) and check_arvhivo_corrida(file,folder_path_caen,num_header_lines,n_channels):
           #procesar los datos adquiridos
           #se procesa un archivo Runxx_list.txt de files_to_proces
           #ejecutar con cada archivo Runxx_list.txt
           file_path=folder_path_caen+'/'+file
           file_to_proces=open(file=file_path,mode='r')
           data_and_header=file_to_proces.readlines()
           file_to_proces.close()
           output_file_path=output_folder_path+'/'+name
           output_file=open(file=output_file_path,mode='a')

           header=data_and_header[:num_header_lines]
           if count_files_run_list_processed==0:
              string_date_time=''
              for i in header[6].split()[4:-1]:i+=' ';string_date_time+=i
              #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
              ref_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')-timedelta(hours=5)
              ref_date_time=pd.Timestamp(str(ref_date_time))
              delta_time=ref_date_time-ref_date_time
              aux=['ch0'+str(i) if i<10 else 'ch'+str(i) for i in range(n_channels)]
              aux2=[aux[i]+' ' if i!=(len(aux)-1) else aux[i] for i in range(len(aux))]
              header2='Tstamp GMT-5(aa-mm-dd hh:mm:ss.ns) '
              for i in aux2:header2+=i
              output_file.write(header2+'\n')
           else:
              string_date_time=''
              for i in header[6].split()[4:-1]:i+=' ';string_date_time+=i
              #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
              last_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')-timedelta(hours=5)
              last_date_time=pd.Timestamp(str(last_date_time))
              delta_time=last_date_time-ref_date_time

           ini_index=num_header_lines+1
           end_index=num_header_lines+n_channels+1
           while ini_index<(len(data_and_header)-1):
                #recorrer todos los data blocks de un archivo Runxx_list.txt
                data_block=data_and_header[ini_index:end_index]
                #ejeutar para cada data block
                ch_counts=[]
                for i in range(len(data_block)):ch_counts.append(data_block[i].split()[-1])
                if ch_counts.count('0')!=n_channels:#verificar que almenos un ch se haya activado
                    Tstamp_us=data_block[0].split()[0]
                    nanosegundos=(delta_time.total_seconds()*1e+9)+(float(Tstamp_us)*1000)
                    Tstamp_gmt_5=ref_date_time+pd.Timedelta(nanosegundos,unit='ns')
                    output_file.write(str(Tstamp_gmt_5))
                    for _ in range(36-len(str(Tstamp_gmt_5))):output_file.write(' ')
                    for i in range(len(data_block)):
                        count=data_block[i].split()[-1]
                        if i!=(len(data_block)-1):
                           output_file.write(count)
                           for _ in range(4+1-len(list(count))):output_file.write(' ')
                        else:
                           output_file.write(count+'\n')
                ini_index=end_index
                end_index=ini_index+n_channels
           output_file.close()
           count_files_run_list_processed+=1
           #os.remove(file_path)#eliminar archivo runxxlits.txt
           del data_and_header#liberar memoria
        if re.search('Info.txt',file):
           #procesar metadata de los datos adquiridos
           pass
    if os.path.exists(output_file_path):
       output_file=open(file=output_file_path,mode='r');content_output_file=output_file.readlines();output_file.close()
       if not content_output_file[1:]:os.remove(output_file_path)#eliminar txt si esta vacio
       del content_output_file
    vars_file=open(file=folder_path_vars,mode='a')
    if processed_file_list:
       #processed_file_list is not empty
       vars_file.write('\n')
       for i in range(0,len(files_to_proces)):
           if i!=(len(files_to_proces)-1):
              vars_file.write(files_to_proces[i]+'\n')
           else:
              vars_file.write(files_to_proces[i])
    else:
      #processed_file_list is empty
      for i in range(0,len(files_to_proces)):
          if i!=(len(files_to_proces)-1):
              vars_file.write(files_to_proces[i]+'\n')
          else:
              vars_file.write(files_to_proces[i])
    vars_file.close()
print('Duracion del procesamiento:',(datetime.now(timezone('America/Bogota'))-ini_exe).total_seconds(),'segundos')
##end main loop

In [ ]:
#generar pixel map